In [17]:
import cv2
import numpy as np
model_name = 'res10_300x300_ssd_iter_140000.caffemodel'
prototxt_name = 'deploy.prototxt.txt'

In [18]:
min_confidence = 0.3
file_name = "image/marathon_01.jpg"

In [19]:
def detectAndDisplay(frame):
    # 카페 모델을 읽어와서 모델객체에 저장합니다
    model = cv2.dnn.readNetFromCaffe(prototxt_name, model_name)
    
    # 이미지 사이즈 조정, 노멀라이징 실행
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), \
                                (104.0, 177.0, 123.0))
    # (104.0, 177.0, 123.0) : RGB 값의 일부를 제외해서 dnn이 분석하기 쉽게 단순화
    # 경험적 최적 수치므로 사용시 값이 거의 고정된 값으로 사용됨
    # 1.0 : 출력 영상의 픽셀값에 곱할 값, 기본값은 1.0
    # (300, 300) : 출력영상의 크기
    
    model.setInput(blob)
    detections = model.forward()  # 얼굴인식에 의한 리턴값 리스트
    
    print(detections.shape)
    print(detections[0])

    # 얻어온 값을 이용하여 반복 실행
    for i in range(0, detections.shape[2]):
        # 대상박스가 얼굴일 확률 추출
        confidence = detections[0,0,i,2]
        # 0.3보다 확률이 높다면 박스 표시 및 확률표시
        if confidence > min_confidence:
            box = detections[0,0,i,3:7] * np.array([width, height, width, height])
            # 3:7 : 3,4,5,6 인덱스 : 박스가 그림상에 위치한 가로세로 상대 위치값
            (startX, startY, endX, endY) = box.astype("int")
            # print(confidence, startX, startY, endX, endY)

            text = "{:.2f}%".format(confidence*100)
            # 박스가 너무 위에 있어 확률표시가 어렵다면 박스 안쪽에 확률표시
            y = startY - 10 if startY- 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0,255,0), 2)
            cv2.putText(frame, text, (startX, y), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.imshow("Face Detection by dnn", frame)

In [20]:
img=cv2.imread(file_name)
print("width: {} pixels".format(img.shape[1]))
print("height: {} pixels".format(img.shape[0]))
print("channels: {}".format(img.shape[2]))
(height, width) = img.shape[:2]
detectAndDisplay(img)
#cv2.imshow("Original Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

width: 1024 pixels
height: 799 pixels
channels: 3


error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\dnn\src\caffe\caffe_io.cpp:1121: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "deploy.prototxt.txt" in function 'cv::dnn::ReadProtoFromTextFile'
